# End of game measures
This `Python` notebook takes the anonymized data and computes population-level measures for each game.

In [2]:
%pylab inline
import json
import numpy as np
import pandas as pd
import glob
import itertools
from sklearn.decomposition import PCA
from scipy import stats

from helpers import shuffle

Populating the interactive namespace from numpy and matplotlib


In [3]:
output_dir = "../results-anonymized/pilot/"
files = glob.glob(output_dir+'block_*_pilot.json')
files

['../results-anonymized/pilot/block_20200505_pilot.json',
 '../results-anonymized/pilot/block_20200507_pilot.json',
 '../results-anonymized/pilot/block_20200624_pilot.json',
 '../results-anonymized/pilot/block_20200626_pilot.json',
 '../results-anonymized/pilot/block_20200506_pilot.json']

In [4]:
blocks = []
for file in files:
    with open(file) as f:
        blocks.append(json.load(f))

In [5]:
# Enumerate clues to be used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']

# Enumerate end-of-game survey questions to be used in polarization analysis
assessments = ['appearance_1', 'appearance_2', 
               'clothing_1', 'clothing_2',
               'suspect_1', 'suspect_2', 'suspect_3',
               'tool_1', 'tool_2', 
               'vehicle_1', 'vehicle_2']    
    
def compute_single_point_measures(game):
    """ 
    Compute the game-level measures 
    
    "Games" in this experiment contain both a treatment and control condition
    and these must be properly separated from one another.
    
    """
    # Form end-of-game survey responses into a dataframe
    collector = {}
    for p, k in game['players'].items():
        try:
            collector[k['data.position']] = k['data.caseMade']
        except:
            print('%s did not complete the post-game survey' %k['data.position'])
    responses = pd.DataFrame(collector).T.sort_index()

    # Form final notebook states into a dataframe
    final_adoptions = pd.DataFrame(data=0, index=responses.index, columns=t_spokes+c_spokes)
    for p, k in game['players'].items():
        for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
            final_adoptions.loc[k['data.position'], clue_id] = 1


    # Determine the number of datapoints to be used in polarization analysis
    # if there are missing responses, need to compare equal sized datasets
    t_responses = [pos for pos in responses.index if pos.startswith('t')]
    c_responses = [pos for pos in responses.index if pos.startswith('c')] 
    # use whichever condition has fewer responses to set the sample size
    n_used = min(len(t_responses), len(c_responses))


    def process_subset(subset, spokes):
        """ compute a result on the selected subset of the data """
        sub_res = {}
        
        # select the subset of the survey responses that will be used in the subset analysis
        sub_survey = responses.loc[subset, assessments]
        
        # survey PC1 
        pca = PCA(n_components=1)
        pca.fit(sub_survey)  
        sub_res['survey PC1'] = pca.explained_variance_ratio_

        # survey similarity percentiles
        survey_corrs = sub_survey.T.corr().mask(np.tri(n_used, n_used, 0, dtype='bool')).stack()
        sub_res['survey 5% similarity'], sub_res['survey 95% similarity'] = np.percentile(
            survey_corrs, [5, 95])
        
        # select the subset of the behavioral responses that will be used in the subset analysis
        sub_adopt = final_adoptions.loc[subset, spokes]
        
        # final-state PC1
        pca = PCA(n_components=1)
        pca.fit(sub_adopt)  
        sub_res['spoke PC1'] = pca.explained_variance_ratio_
        
        # final state similarity percentiles
        spoke_corrs = sub_adopt.T.corr().mask(np.tri(n_used, n_used, 0, dtype='bool')).stack()
        sub_res['spoke 5% similarity'], sub_res['spoke 95% similarity'] = np.percentile(
            spoke_corrs, [5, 95])
        
        # compute the expected values for the given level of adoption
        # by shuffling the clues between individuals 
        # (preserving the number of clues each individual holds, 
        # and the number of individuals holding each clue)
        # do this a number of times and average the result
        e95 = []
        e5 = []
        ePC1 = []
        for _ in range(100):
            shuffle_adopt = pd.DataFrame(index=sub_adopt.index,
                                         columns=sub_adopt.columns,
                                         data=shuffle(sub_adopt.values, n=500))

            n_agents = len(shuffle_adopt.index)
            corrs = shuffle_adopt.astype(float).T.corr().mask(np.tri(n_agents, n_agents, 0, dtype='bool')).stack()
            e95.append(np.percentile(corrs, 95))
            e5.append(np.percentile(corrs, 5))

            pca = PCA(n_components=1)
            pca.fit(shuffle_adopt)
            ePC1.append(pca.explained_variance_ratio_[0])
        
        # compute the net effect of (interdependent or independent) diffusion 
        # over chance distribution of the same clues
        sub_res['net spoke PC1'] = sub_res['spoke PC1'] - np.mean(ePC1)
        sub_res['net spoke 95% similarity'] = sub_res['spoke 95% similarity'] - np.mean(e95)
        sub_res['net spoke 5% similarity'] = sub_res['spoke 5% similarity'] - np.mean(e5)
        
        return sub_res
        
        
    # For each subset of size 'n_used', compute a result. 
    # In most cases there are no missing responses, so just compute on the complete set
    t_collector = []
    for subset in itertools.combinations(t_responses, r=n_used):
        t_collector.append(process_subset(subset, t_spokes))

    # The recorded result is the average over all subsets
    if len(t_collector) > 1:
        print('Averaging over %i combinations for treatment case'%len(t_collector))
    t_result = pd.DataFrame(t_collector).mean()

    # Compute average for confidence and consensus measures on all submissions
    t_result['confidence'] = responses.loc[t_responses, 'confidence'].mean()
    t_result['consensus'] = responses.loc[t_responses, 'consensus'].mean()    

    
    # Perform the same analysis as above for the control condition
    c_collector = []
    for subset in itertools.combinations(c_responses, r=n_used):
        c_collector.append(process_subset(subset, c_spokes))

    if len(c_collector) > 1:
        print('Averaging over %i combinations for control case'%len(c_collector))
    c_result = pd.DataFrame(c_collector).mean()

    c_result['confidence'] = responses.loc[c_responses, 'confidence'].mean()
    c_result['consensus'] = responses.loc[c_responses, 'consensus'].mean()     

    
    #pd.merge(t_result, c)result, suffixes=(' (inter)', ' (indep)'))
    result = pd.concat([t_result, c_result], keys=['inter', 'indep'])
    #result['game_id']=game['createdAt'].split('_')[0].replace('-','_').replace(':','_').replace('.','_')
    return result

def compute_block(block):
    results_collector = []
    network_collector = []
    for name, game in block.items():
        network_collector.append('caveman' if 'caveman' in name else 'dodec')
        results_collector.append(compute_single_point_measures(game))
        
    result = pd.concat(results_collector, keys=network_collector)
    return result

In [6]:
measurements = pd.concat([compute_block(block) for block in blocks], axis=1)
measurements

0          1          2  \
caveman indep confidence                53.400000  57.550000        NaN   
              consensus                 55.150000  54.550000        NaN   
              net spoke 5% similarity   -0.189744  -0.207843        NaN   
              net spoke 95% similarity   0.315424   0.309143        NaN   
              net spoke PC1              0.206725   0.238717        NaN   
              spoke 5% similarity       -0.418330  -0.313527        NaN   
              spoke 95% similarity       0.806832   0.900423        NaN   
              spoke PC1                  0.382635   0.422482        NaN   
              survey 5% similarity      -0.725347  -0.621686        NaN   
              survey 95% similarity      0.747789   0.723486        NaN   
              survey PC1                 0.359687   0.440281        NaN   
        inter confidence                53.500000  52.350000        NaN   
              consensus                 51.600000  57.100000        NaN   
              net spoke 5% similarity   -0.139049  -0.174164        NaN   
              net spoke 95% similarity   0.378327   0.294375        NaN   
              net spoke PC1              0.316963   0.212154        NaN   
              spoke 5% similarity       -0.340905  -0.430331        NaN   
              spoke 95% similarity       0.896421   0.783046        NaN   
              spoke PC1                  0.505600   0.397125        NaN   
              survey 5% similarity      -0.553209  -0.539193        NaN   
              survey 95% similarity      0.873783   0.686564        NaN   
              survey PC1                 0.488426   0.354293        NaN   
dodec   indep confidence                43.350000  50.500000  50.000000   
              consensus                 42.700000  50.600000  54.350000   
              net spoke 5% similarity   -0.172933  -0.143391  -0.010965   
              net spoke 95% similarity   0.195880   0.141552   0.045216   
              net spoke PC1              0.090788   0.116922   0.035288   
              spoke 5% similarity       -0.293518  -0.318754  -0.157895   
              spoke 95% similarity       0.717430   0.670820   0.690066   
              spoke PC1                  0.342774   0.374724   0.252950   
              survey 5% similarity      -0.492309  -0.485883  -0.209945   
              survey 95% similarity      0.575030   0.739458   0.780277   
              survey PC1                 0.279186   0.419603   0.267906   
        inter confidence                59.350000  58.300000  48.850000   
              consensus                 53.500000  54.450000  53.200000   
              net spoke 5% similarity   -0.156672  -0.126372  -0.030345   
              net spoke 95% similarity   0.178617   0.014803   0.093935   
              net spoke PC1              0.118987   0.056466   0.042546   
              spoke 5% similarity       -0.334691  -0.138534  -0.216025   
              spoke 95% similarity       0.716328   0.642071   0.651795   
              spoke PC1                  0.301137   0.249915   0.253256   
              survey 5% similarity      -0.549573  -0.351835  -0.489567   
              survey 95% similarity      0.746668   0.690122   0.650281   
              survey PC1                 0.417522   0.311937   0.265225   

                                                3          4  
caveman indep confidence                      NaN  45.900000  
              consensus                       NaN  48.600000  
              net spoke 5% similarity         NaN  -0.128200  
              net spoke 95% similarity        NaN   0.215667  
              net spoke PC1                   NaN   0.152079  
              spoke 5% similarity             NaN  -0.294525  
              spoke 95% similarity            NaN   0.803894  
              spoke PC1                       NaN   0.407205  
              survey 5% similarity            NaN  -0.490766  
              survey 95% similarity       

In [31]:
def bootstrap_mean(l, q=(2.5, 97.5), n=1000):
    "Basic bootstrap confidence intervals (q) with n resamples"
    return np.percentile([np.mean(np.random.choice(l, size=len(l))) for _ in range(n)], q=q)


def mean_result(measure1, _):
    return measure1.mean()

def mean_lowerbound(measure1, _):
    return bootstrap_mean(measure1)[0]

def mean_upperbound(measure1, _):
    return bootstrap_mean(measure1)[1]

def effect_size(measure1, measure2):
    return (measure1 - measure2).mean()

def effect_lowerbound(measure1, measure2):
    return bootstrap_mean(measure1 - measure2)[0]

def effect_upperbound(measure1, measure2):
    return bootstrap_mean(measure1 - measure2)[1]

def effect_p_val(measure1, measure2):
    return stats.ttest_rel(measure1, measure2)[1]


def make_table(measurements, func):
    rows = measurements.index.levels[2]
    cols = np.unique(measurements.index.droplevel(2))
    res = pd.DataFrame(index=rows, columns=cols)
    for row in rows:
        for col in cols:
            comparison = measurements.loc[col+tuple([row])].dropna()
            reference = measurements.loc[('dodec', 'indep')+tuple([row])][comparison.index]
            res.at[row, col] = func(comparison, reference)
    return res

In [32]:
res = make_table(measurements, mean_result)
res

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,52.2833,56.7667,48.99,53.38
consensus,52.7667,58.3333,49.07,53.02
net spoke 5% similarity,-0.175262,-0.121668,-0.068507,-0.0969845
net spoke 95% similarity,0.280078,0.382334,0.117033,0.0996177
net spoke PC1,0.199174,0.245981,0.0585099,0.0763013
spoke 5% similarity,-0.342128,-0.353304,-0.177564,-0.245504
spoke 95% similarity,0.83705,0.893156,0.698852,0.646924
spoke PC1,0.404108,0.43493,0.299288,0.269057
survey 5% similarity,-0.6126,-0.556754,-0.395819,-0.469243
survey 95% similarity,0.729252,0.788166,0.711058,0.683996


In [33]:
res_lowerbound = make_table(measurements, mean_lowerbound)
res_lowerbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,45.9,52.35,45.17,48.22
consensus,48.6,51.6,44.06,51.6188
net spoke 5% similarity,-0.207843,-0.174164,-0.129061,-0.13849
net spoke 95% similarity,0.215667,0.294375,0.0702829,0.0477121
net spoke PC1,0.152079,0.208825,0.0226683,0.0529869
spoke 5% similarity,-0.41833,-0.430331,-0.281535,-0.323293
spoke 95% similarity,0.803894,0.783046,0.645074,0.618184
spoke PC1,0.382635,0.397125,0.253144,0.252542
survey 5% similarity,-0.725347,-0.57786,-0.510874,-0.543157
survey 95% similarity,0.71648,0.686564,0.63834,0.649199


In [35]:
res_upperbound = make_table(measurements, mean_upperbound)
res_upperbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,57.55,60.8912,52.54,58
consensus,55.15,66.3,54.08,54.06
net spoke 5% similarity,-0.1282,-0.05179,-0.00204417,-0.0525156
net spoke 95% similarity,0.315424,0.4743,0.160681,0.149317
net spoke PC1,0.238717,0.316963,0.0933706,0.102005
spoke 5% similarity,-0.294525,-0.288675,-0.0696658,-0.162739
spoke 95% similarity,0.900423,1,0.758169,0.681922
spoke PC1,0.422482,0.5056,0.345432,0.288255
survey 5% similarity,-0.490766,-0.539193,-0.275828,-0.394175
survey 95% similarity,0.747789,0.873783,0.759027,0.721845


In [36]:
effect = make_table(measurements, effect_size)
effect

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,2.4,6.88333,0,4.39
consensus,3.15,8.71667,0,3.95
net spoke 5% similarity,-0.0557543,-0.00215941,0,-0.0284775
net spoke 95% similarity,0.124833,0.22709,0,-0.0174151
net spoke PC1,0.130216,0.177023,0,0.0177914
spoke 5% similarity,-0.14851,-0.159686,0,-0.0679407
spoke 95% similarity,0.106442,0.162549,0,-0.0519284
spoke PC1,0.0675537,0.0983758,0,-0.030231
survey 5% similarity,-0.198728,-0.142882,0,-0.0734243
survey 95% similarity,0.0466303,0.105545,0,-0.0270628


In [42]:
effect_pval = make_table(measurements.dropna(axis=1), p_val)
effect_pval

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,0.736415,0.114478,NaN,0.229662
consensus,0.631243,0.0193449,NaN,0.575628
net spoke 5% similarity,0.112237,0.920025,NaN,0.26246
net spoke 95% similarity,0.0331431,0.0633835,NaN,0.168621
net spoke PC1,0.00767666,0.0500666,NaN,0.784493
spoke 5% similarity,0.263095,0.191935,NaN,0.870416
spoke 95% similarity,0.251753,0.0240253,NaN,0.3424
spoke PC1,0.105343,0.138205,NaN,0.143659
survey 5% similarity,0.0242238,0.237789,NaN,0.846876
survey 95% similarity,0.536669,0.413217,NaN,0.69924


In [39]:
effect_lowerbound = make_table(measurements, effect_lowerbound)
effect_lowerbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,-9.9,1.85,0,-0.71
consensus,-6.95,6.5,0,-1.83
net spoke 5% similarity,-0.0860001,-0.0307731,0,-0.0924617
net spoke 95% similarity,0.0873654,0.152824,0,-0.0780243
net spoke PC1,0.115937,0.0952319,0,-0.0249942
spoke 5% similarity,-0.325946,-0.320095,0,-0.180608
spoke 95% similarity,0.00032269,0.112226,0,-0.126495
spoke PC1,0.039861,0.022401,0,-0.0819836
survey 5% similarity,-0.233037,-0.314437,0,-0.196981
survey 95% similarity,-0.0168953,-0.0528939,0,-0.101664


In [38]:
effect_upperbound = make_table(measurements, effect_upperbound)
effect_upperbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,10.05,10.15,0,10.93
consensus,12.45,10.75,0,9.73
net spoke 5% similarity,-0.0168106,0.0338849,0,0.0125661
net spoke 95% similarity,0.167591,0.345999,0,0.0427601
net spoke PC1,0.152916,0.226174,0,0.0512252
spoke 5% similarity,0.00522697,-0.0473868,0,0.0564639
spoke 95% similarity,0.229602,0.196429,0,-0.0026986
spoke PC1,0.115042,0.162826,0,0.0149828
survey 5% similarity,-0.135803,-0.0533101,0,0.0433194
survey 95% similarity,0.172759,0.298753,0,0.073216


In [44]:
export_table = pd.DataFrame()
export_table["Result Mean"] = res.unstack()
export_table["Result Lower Bound"] = res_lowerbound.unstack()
export_table["Result Upper Bound"] = res_upperbound.unstack()
export_table["Effect Size"] = effect.unstack()
export_table["Effect Upper Bound"] = effect_upperbound.unstack()
export_table["Effect Lower Bound"] = effect_lowerbound.unstack()
export_table["Effect P Value"] = effect_pval.unstack()


export_table.to_csv(output_dir+"end_of_game_measures.csv")
export_table

Result Mean Result Lower Bound  \
(caveman, indep) confidence                   52.2833               45.9   
                 consensus                    52.7667               48.6   
                 net spoke 5% similarity    -0.175262          -0.207843   
                 net spoke 95% similarity    0.280078           0.215667   
                 net spoke PC1               0.199174           0.152079   
                 spoke 5% similarity        -0.342128           -0.41833   
                 spoke 95% similarity         0.83705           0.803894   
                 spoke PC1                   0.404108           0.382635   
                 survey 5% similarity         -0.6126          -0.725347   
                 survey 95% similarity       0.729252            0.71648   
                 survey PC1                  0.368261           0.304816   
(caveman, inter) confidence                   56.7667              52.35   
                 consensus                    58.3333               51.6   
                 net spoke 5% similarity    -0.121668          -0.174164   
                 net spoke 95% similarity    0.382334           0.294375   
                 net spoke PC1               0.245981           0.208825   
                 spoke 5% similarity        -0.353304          -0.430331   
                 spoke 95% similarity        0.893156           0.783046   
                 spoke PC1                    0.43493           0.397125   
                 survey 5% similarity       -0.556754           -0.57786   
                 survey 95% similarity       0.788166           0.686564   
                 survey PC1                  0.423766           0.354293   
(dodec, indep)   confidence                     48.99              45.17   
                 consensus                      49.07              44.06   
                 net spoke 5% similarity    -0.068507          -0.129061   
                 net spoke 95% similarity    0.117033          0.0702829   
                 net spoke PC1              0.0585099          0.0226683   
                 spoke 5% similarity        -0.177564          -0.281535   
                 spoke 95% similarity        0.698852           0.645074   
                 spoke PC1                   0.299288           0.253144   
                 survey 5% similarity       -0.395819          -0.510874   
                 survey 95% similarity       0.711058            0.63834   
                 survey PC1                  0.318089           0.264056   
(dodec, inter)   confidence                     53.38              48.22   
                 consensus                      53.02            51.6188   
                 net spoke 5% similarity   -0.0969845           -0.13849   
                 net spoke 95% similarity   0.0996177          0.0477121   
                 net spoke PC1              0.0763013          0.0529869   
                 spoke 5% similarity        -0.245504          -0.323293   
                 spoke 95% similarity        0.646924           0.618184   
                 spoke PC1                   0.269057           0.252542   
                 survey 5% similarity       -0.469243          -0.543157   
                 survey 95% similarity       0.683996           0.649199   
                 survey PC1                  0.342388           0.293604   

                                          Result Upper Bound Effect Size  \
(caveman, indep) confidence                            57.55         2.4   
                 consensus                             55.15        3.15   
                 net spoke 5% similarity             -0.1282  -0.0557543   
                 net spoke 95% similarity           0.315424    0.124833   
                 net spoke PC1                      0.238717    0.130216   
                 spoke 5% similarity               -0.294525    -0.14851   
                 spoke 95% similarity               0.900423    0.106442   
             